In [12]:
import numpy as np
import pandas as pd
import subprocess
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
from IPython.display import clear_output

In [3]:
prqt_loc = '../../../../datasets/ess/parquet/'

In [4]:
p = subprocess.Popen(f'ls {prqt_loc}', shell=True, stdout=subprocess.PIPE)
ret = p.communicate()
print('Exit code: {}\nOutput:\n\n{}'.format(p.returncode, ret[0].decode('utf-8')))

Exit code: 0
Output:

ESS1_180115.parquet
ESS2_180115.parquet
ESS3_180115.parquet
ESS4_190213.parquet
ESS5_180115.parquet
ESS6_180115.parquet
ESS7_180115.parquet
ESS8_190213.parquet



In [5]:
parquets = [ pqt for pqt in ret[0].decode('utf-8').split('\n') if pqt.find('.parquet') > 0 ]

In [6]:
df_dtypes = pd.DataFrame()
for parquet in parquets:
    df = pd.read_parquet(f"{prqt_loc}{parquet}")
    dtype_dict = df.dtypes.to_dict()
    for key in dtype_dict:
        df_dtypes.loc[parquet,key] = dtype_dict[key]

In [7]:
ess_schema_new = {}
for col in df_dtypes.columns:
    if df_dtypes[col].isna().sum() > 0:
        if df_dtypes[col].value_counts().index[0] == np.dtype('O'):
            ess_schema_new[col] = np.dtype('O')
        else:
            ess_schema_new[col] = np.dtype('float64')
    else:
        ess_schema_new[col] = df_dtypes[col].value_counts().index[0]

In [8]:
cols_naint = []
for col in ess_schema_new.keys():
    if ess_schema_new[col] == np.dtype('int64'):
        if df[col].isna().sum() > 0:
            cols_naint.append(col)

In [9]:
for col in cols_naint:
    ess_schema_new[col] = np.dtype('float64')

In [10]:
for parquet in parquets:
    #test and repair ess_schema_new
    print(parquet)
    df = pd.read_parquet(f"{prqt_loc}{parquet}")
    diff_col = list( set(ess_schema_new.keys()) - set(df.columns) )
    df2 = pd.DataFrame()
    df2 = pd.DataFrame(columns = [col for col in diff_col])
    df2 = pd.concat([df,df2],sort=False)
    for col in list(ess_schema_new.keys()):
        try:
            df2[col] = df2[col].astype( ess_schema_new[col] , copy=False)
        except:
            value_dict = {}
            for value in df2[col].value_counts().index:
                if type(value) in value_dict.keys():
                    value_dict[type(value)] += 1
                else:
                    value_dict[type(value)] = 1
            if sorted( list( value_dict.keys()  ), key= lambda x: -value_dict.get(x)  )[0] == str:
                ess_schema_new[col] = np.dtype('O')
            else:
                ess_schema_new[col] = np.dtype('float64')
            print('##repair schema',col)

ESS1_180115.parquet
##repair schema ctzship
##repair schema ctzshipa
##repair schema cntbrth
##repair schema cntbrtha
ESS2_180115.parquet
##repair schema fbrncnt
##repair schema mbrncnt
ESS3_180115.parquet
ESS4_190213.parquet
##repair schema ctzshipb
##repair schema cntbrthb
##repair schema fbrncnta
##repair schema mbrncnta
ESS5_180115.parquet
ESS6_180115.parquet
ESS7_180115.parquet
ESS8_190213.parquet


In [13]:
for parquet in parquets:
    #Rebuild parquets
    print(parquet)
    df = pd.read_parquet(f"{prqt_loc}{parquet}")
    diff_col = list( set(ess_schema_new.keys()) - set(df.columns) )
    df2 = pd.DataFrame()
    df2 = pd.DataFrame(columns = [col for col in diff_col])
    df2 = pd.concat([df,df2],sort=False)
    for col in list(ess_schema_new.keys()):
        df2[col] = df2[col].astype( ess_schema_new[col] , copy=False)
    #pandas to arrwo type differences
    pta_td = {np.dtype('float64'):pa.float64(),
                      np.dtype('int64'):pa.int64(),
                      np.dtype('datetime64[ns]'):pa.timestamp('ns'),
                      np.dtype('O'):pa.string()}
    schema = pa.schema([ pa.field(key, pta_td[ ess_schema_new[key] ], nullable=True) for key in ess_schema_new.keys() ])
    table = pa.Table.from_pandas(df2[list(ess_schema_new.keys())], schema=schema,preserve_index=False)
    pq.ParquetWriter(f"{prqt_loc}ns_{parquet}", table.schema).write_table(table)

ESS1_180115.parquet
ESS2_180115.parquet
ESS3_180115.parquet
ESS4_190213.parquet
ESS5_180115.parquet
ESS6_180115.parquet
ESS7_180115.parquet
ESS8_190213.parquet
